In [1]:
#- 패키지 임포트
import pandas as pd
import numpy as np
#import emoji # 이모지 제거용
import re
from soynlp.normalizer import * #반복되는 자음 제거
import time
from tqdm.notebook import tqdm #for문 진행률 
from time import sleep #for문 진행 시간 
from kiwipiepy import Kiwi #문장 분리2

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)


Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("./산림복지/2019산림복지_본문_노이즈 제거.txt", sep='\t', encoding='UTF-8')
# print(df.shape, type(df)) # 파일 구조 및 컬럼명 확인용
df['org_idx']= df.index # 인덱스 column 만들기: 문장 분리할 때 사용함
# print(df.columns) # org_idx 제대로 추가됐나 단순 확인용.
post_list = list(df['본문']) #자연어 전처리를 위해 본문을 리스트 형태로

In [3]:
''' 불용어 제거 '''
# post_list 양이 많을 경우 불용어 제거 중에 오류 나면 다시 리스트로 만드는 데 오래 걸려서 카피하는 것임. 안 하고 post_list로 진행해도 됨.
post_list2 = post_list.copy()
for i in range(len(post_list2)):
    post_list2[i] = re.sub('\n', ' ', string=str(post_list2[i])) # 줄바꿈을 띄어쓰기 하나로
    post_list2[i] = re.compile('[^|ㄱ-ㅎ|ㅏ-ㅣ|A-Z|a-z|0-9|가-힣]+').sub(' ',post_list2[i])
    post_list2[i] = repeat_normalize(post_list2[i], num_repeats=1)
	# ↑ ㅋㅋㅋ, ㅎㅎㅎ 등의 의미 없는 자음 삭제. num_repeats가 1이면 반복 글자 중 1개만 남기고 2면 2개만 남김.
    post_list2[i] = re.sub(r"^\s+|\s+$", "", post_list2[i]) # 문서 앞뒤 공백 제거
    post_list2[i] = post_list2[i].split() # 문서 내 공백(1개 이상) 기준으로 자르기
    post_list2[i] = ' '.join(post_list2[i]) # 공백 기준으로 나뉜 문서를 다시 1개의 공백을 두고 붙임.
	# ↑ 문서 내 다중 공백을 지우기 위함.]

df['cleaning'] = pd.DataFrame(post_list2) #불용어 제거된 본문을 데이터 프레임으로

In [ ]:
''' 문장 분리 '''
kiwi = Kiwi() #kss보다 속도 빠름 
split_list = [kiwi.split_into_sents(df['cleaning'][i]) for i in tqdm(range(len(df['cleaning'])))]
df['split_list'] = split_list
df

In [2]:
df = pd.read_csv("2019산림복지_세이브포인트.csv", encoding='UTF-8')
df

,url,title,date,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/waura21/221755822694,[코리아투데이뉴스=상주시] 상주시 2020년 상반기 정기인사...,2019.12.31.,[코리아투데이뉴스=상주시] 상주시 2020년 상반기 정기인사 이동조서-(2020. ...,0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,[Sentence(text='코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인...
1,https://blog.naver.com/skfzhddlwkd/221755736686,[지방분권 대학생 서포터즈] 공공기관 지방이전과 혁신도시,2019.12.31.,안녕하세요~ 메리썽입니다.\n혁신도시에 대해서는 많이들 들어보셨을거라고 생각이 드는...,1,안녕하세요 메리썽입니다 혁신도시에 대해서는 많이들 들어보셨을거라고 생각이 드는데요 ...,"[Sentence(text='안녕하세요 메리썽입니다', start=0, end=12..."
2,https://blog.naver.com/vipgim1390/221755564191,[인사] 경북 구미시,2019.12.31.,◇4급 승진 △경제기획국장 박수원 △문화체육관광국장 김회식 △의회사무국장 이근도 △...,2,4급 승진 경제기획국장 박수원 문화체육관광국장 김회식 의회사무국장 이근도 평생교육원...,[Sentence(text='4급 승진 경제기획국장 박수원 문화체육관광국장 김회식 ...
3,https://blog.naver.com/aromaplayer/221755469475,경기창조경제혁신센터_로컬크리에이터_사후간담회_역시...,2019.12.31.,안녕하세요!\n반고 유은기(유진선)입니다.\n오늘은 2019년의 마지막날!! 입니다...,3,안녕하세요 반고 유은기 유진선 입니다 오늘은 2019년의 마지막날 입니다 올해는 정...,"[Sentence(text='안녕하세요 반고 유은기 유진선 입니다', start=0..."
4,https://blog.naver.com/sejongicon/221755302232,국립칠곡숲체원 사인/ 간판디자인,2019.12.31.,한국산림복지진흥원 국립칠곡숲체원 사인/ 간판 디자인 및 시공\nhttps://blo...,4,한국산림복지진흥원 국립칠곡숲체원 사인 간판 디자인 및 시공 https blog na...,[Sentence(text='한국산림복지진흥원 국립칠곡숲체원 사인 간판 디자인 및 ...
...,...,...,...,...,...,...,...
6122,https://blog.naver.com/vipgim1390/221431934615,[인사] 경북 김천시,2019.01.01.,◆4급 승진 △금오공대 파견 김용수 △경제산업국장 구영훈 △의회사무국장 안민 △행정...,6122,4급 승진 금오공대 파견 김용수 경제산업국장 구영훈 의회사무국장 안민 행정복지국장 ...,[Sentence(text='4급 승진 금오공대 파견 김용수 경제산업국장 구영훈 의...
6123,https://blog.naver.com/dltmf1427/221431893814,"2018년 마무리, 2019년 새해 목표는 세우셨나요? (부록...",2019.01.01.,안녕하세요.\n라포르커뮤니케이션 원쌤이에요.\n2019년 한 해 잘 시작하셨나요^^...,6123,안녕하세요 라포르커뮤니케이션 원쌤이에요 2019년 한 해 잘 시작하셨나요 나이 먹기...,"[Sentence(text='안녕하세요 라포르커뮤니케이션 원쌤이에요', start=..."
6124,https://blog.naver.com/rich2s/221431887654,2018년 12월 31일 동·서·남해안 및 내륙권 발전 특별법...,2019.01.01.,블로그\n부동산뉴스 95개의 글\n부동산뉴스\n목록열기\n2018년 12월 31일 ...,6124,블로그 부동산뉴스 95개의 글 부동산뉴스 목록열기 2018년 12월 31일 동 서 ...,[Sentence(text='블로그 부동산뉴스 95개의 글 부동산뉴스 목록열기 20...
6125,https://blog.naver.com/changsantree/221431590702,1월 1일 화요일,2019.01.01.,blog\n공지 목록\n공지글\n글 제목 작성일\n(1)\n공지 3월 27일 수요일...,6125,blog 공지 목록 공지글 글 제목 작성일 1 공지 3월 27일 수요일 2019 3...,[Sentence(text='blog 공지 목록 공지글 글 제목 작성일 1 공지 3...


In [4]:
df["split_list"][0]

"[Sentence(text='코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 6급 공보감사담당관 지역정보팀장 김귀연 미래전략추진단 인구정책팀장 이상규 총무과 총무팀장 조성광 인사팀장 신종원 교육후생팀장 김상식 송상섭 민순희 새마을체육과 시민운동장팀장 김호웅 회계과 지출팀장 남정억 재산관리팀장 이상익 사회복지과 희망복지지원팀장 서명옥 가족복지과 노인복지팀장 강정화 경제기업과 지역경제팀장 우광하 경제기업과 기업지원팀장 윤태경 환경관리과 폐기물관리팀장 김정진 농업정책과 농산팀장 김영기 농업정책과 원예특작팀장 최정숙 산림녹지과 산림정책팀장 김대환 산림복지팀장 안재현 곶감관리팀장 조준홍 건설과 토목팀장 김시진 하천팀장 김영국 개발지원과 개발행정팀장 홍헌식 시설지원팀장 이두희 안전재난과 재난방재팀장 이춘길 안전재난과 민방위팀장 이하국 교통에너지과 교통지도팀장 정원용 차량등록팀장 성일중 자전거팀장 김광현 건축과 건축허가팀장 김덕수 건축신고1팀장 김형수 건축신고2팀장 신은우 주택팀장 김웅정 민원토지과 개발행위팀장 김동구 가족관계등록팀장 김명원 의회사무국 고두환 보건소보건위생과 보건행정팀장 김기우 김근우 김혜경 남정선 문정옥 배완식 안윤정 우상하 위외자 정해숙 조성욱 한현정 강옥분 김승란 손미자 이성숙 정안순 피정주 상하수도사업소 상하수도행정팀장 황갑주 마을상수도팀장 이상보 평생학습원 운영지원팀장 박종욱 한방산업단지관리사업소 휴양림팀장 김기완 상주박물관 자전거박물관팀장 김원기 함창읍 안재정 이상문 사벌국면 부면장 박정소 강만식 김경민 정정미 중동면 김학진 박진규 낙동면 박진호 청리면 부면장 박광호 외남면 부면장 정해수 내서면 오종철 모동면 오장엽 모서면 부면장 윤성렬 이기학 화동면 지영진 조용준 화북면 권애경 외서면 윤동규 은척면 송금화 공검면 김주연 이안면 장사준 화남면 윤용운 남원동 조성덕 계림동 전주화 동문동 김민용 김미정 동성동 김범정 신흥동 신영만 6급 무보직 및 7급 이하 기획예산담당관 윤승민 김인욱 박지혜 이진영 성창엽 공보감사

In [6]:
df_1 = df[:1000]
df_2 = df[1000:2000]
df_3 = df[2000:3000]
df_4 = df[3000:4000]
df_5 = df[4000:5000]
df_6 = df[5000:]

In [7]:
# ''' 분리된 문장 한 개를 한 row로 '''
# df2 = pd.DataFrame(columns = ['split_str', 'org_idx']) #용량을 줄이기 위해 새로운 데이터프레임으로

# ''' 문장 데이터 프레임 저장 '''
# df2.to_csv("./산림복지/2019산림복지_본문_전처리.csv", index=False, encoding='UTF-8')

In [8]:
df2 = pd.read_csv("./산림복지/2019산림복지_본문_전처리.csv", encoding='UTF-8')
df2

,split_str,org_idx


In [9]:
df_1['split_list'].index

RangeIndex(start=0, stop=1000, step=1)

In [ ]:
for i in tqdm(df_1['split_list'].index):
    for j in range(len(df_1['split_list'][i])):
        split_str= {'split_str': df_1['split_list'][i][j][0], 'org_idx': [i]}
        print(split_str['org_idx'])
        split_df= pd.DataFrame(split_str)
        df2 = pd.concat([df2,split_df])
        df2.reset_index(drop=True, inplace=True)
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)
df2

In [ ]:
for i in tqdm(df_2['split_list'].index):
    for j in range(len(df_2['split_list'][i])):
        split_str= {'split_str': df_2['split_list'][i][j][0], 'org_idx': [i]}
        print(split_str['org_idx'])
        split_df= pd.DataFrame(split_str)
        df2 = pd.concat([df2,split_df])
        df2.reset_index(drop=True, inplace=True)
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)
df2

In [52]:
for i in tqdm(df_3['split_list'].index):
    for j in range(len(df_3['split_list'][i])):
        split_str= {'split_str': df_3['split_list'][i][j][0], 'org_idx': [i]}
        print(split_str['org_idx'])
        split_df= pd.DataFrame(split_str)
        df2 = pd.concat([df2,split_df])
        df2.reset_index(drop=True, inplace=True)
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)

[2999]


In [ ]:
for i in tqdm(df_4['split_list'].index):
    for j in range(len(df_4['split_list'][i])):
        split_str= {'split_str': df_4['split_list'][i][j][0], 'org_idx': [i]}
        print(split_str['org_idx'])
        split_df= pd.DataFrame(split_str)
        df2 = pd.concat([df2,split_df])
        df2.reset_index(drop=True, inplace=True)
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)
df2

In [ ]:
for i in tqdm(df_5['split_list'].index):
    for j in range(len(df_5['split_list'][i])):
        split_str= {'split_str': df_5['split_list'][i][j][0], 'org_idx': [i]}
        print(split_str['org_idx'])
        split_df= pd.DataFrame(split_str)
        df2 = pd.concat([df2,split_df])
        df2.reset_index(drop=True, inplace=True)
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)
df2

In [ ]:
for i in tqdm(df_6['split_list'].index):
    for j in range(len(df_6['split_list'][i])):
        split_str= {'split_str': df_6['split_list'][i][j][0], 'org_idx': [i]}
        print(split_str['org_idx'])
        split_df= pd.DataFrame(split_str)
        df2 = pd.concat([df2,split_df])
        df2.reset_index(drop=True, inplace=True)
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)
df2

In [21]:
df2.to_csv("./산림복지/2019산림복지_본문_전처리.csv", index=False, encoding='UTF-8')

In [14]:
df2 # 3999까지 한거.

,split_str,org_idx
0,블로그 momo 5개의 글 momo 목록열기 영어명언 모음입니다,0
1,momo 2018 12 31 19 52 https blog naver com omj...,0
2,An enemy generally says and believes what he w...,0
3,True love is the joy of life 진실한 사랑은 인생의 환희다,0
4,Carpe diem 현재를 즐겨라,0
...,...,...
110259,김종욱 이사장은 상대적으로 산림복지서비스 기회가 적은 취약계층에게 숲을 통한 소통 ...,3998
110260,종합지 대한뉴스 등록번호 서울가361호 크리에이티브 코리아 등록번호 강서라00175...,3998
110261,Tag 저작권자 대한뉴스 무단전재 및 재배포 금지 공감 2 이 글에 공감한 블로거 ...,3998
110262,세월속에 세월속에 139개의 글 세월속에 목록열기 목민심서12편72조 0414 세월...,3999


In [ ]:
#- 패키지 임포트
import requests
import json
import pandas as pd
from tqdm.notebook import tqdm
import re


'''맞춤법 검사 함수'''
def ko_grammar(sent):
    global correct_list ; correct_list= []
    #for i in tqdm(range(len(a))):
    for i in tqdm(range(len(sent))):
        try:                                                                               
            response = requests.post('http://164.125.7.61/speller/results', data={'text1': sent[i]})  # 맞춤법 검사 요청 (requests)    
            data = response.text.split('data = [', 1)[-1].rsplit('];', 1)[0] # 응답에서 필요한 내용 추출 (html 파싱)
            data = json.loads(data) # 파이썬 딕셔너리 형식으로 변환
            orgStr = [err['orgStr'] for err in data['errInfo']] #오류가 담긴 errinfo에서 고쳐야 되는 단어 리스트로 추출 
            correct = [err['candWord'] for err in data['errInfo']] #고쳐진 단어도 리스트로 추출
            splited = sent[i] 

            '''여러 개로 고쳐졌을 때 맨 처음 단어만 선택'''
            for j in range(len(correct)):
                if '|'in correct[j]: #여러 개로 고쳐질 경우 문자열이 '|'로 구분되어 있음
                    com = re.compile('\|')
                    many = com.search(correct[j])
                    correct[j]= correct [j][0:(many.span()[0])]
                '''틀린 문장 고치기'''   
                splited = splited.replace(orgStr[j], correct[j])

        except:      
            splited = sent[i]

        correct_list.append(splited)

# 데이터 프레임 나누기
df2_1 = df2[:5000]
df2_2 = df2[5000:10000]
df2_3 = df2[10000:15000]

df2_n = df2[:]

'''인덱스 0부터'''
df2_1.reset_index(drop=True, inplace=True)
df2_2.reset_index(drop=True, inplace=True)
df2_3.reset_index(drop=True, inplace=True)

df2_n.reset_index(drop=True, inplace=True)


'''맞춤법 검사하기''' 
ko_grammar(df2_1['split_str'])
df2_n['correct_str'] = correct_list
df2_n.to_csv('df2_n.txt', encoding='UTF-8')